# DataBase - PSQL

In [1]:
import csv
import io
import psycopg2 as psql
from datetime import datetime
from urllib import request

/home/local/Desktop/py/venvs/ds/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


 stars (*****) - refers to your password

In [2]:
conn = psql.connect(dbname = 'postgres', user = 'postgres', password = 'local')

In [3]:
cur = conn.cursor()

In [4]:
conn.autocommit = True

In [5]:
cur.execute('CREATE DATABASE h_data')

In [6]:
cur.execute("CREATE USER vipul WITH PASSWORD 'local'")

In [7]:
cur.execute("CREATE USER other WITH PASSWORD 'self'")

In [8]:
conn = psql.connect(dbname = 'h_data', user = 'postgres', password = 'local')

In [9]:
conn.autocommit = True

In [10]:
cur = conn.cursor()

In [11]:
s = """
CREATE TABLE data (
FID INTEGER PRIMARY KEY,
TIME TIMESTAMP,
BTID INTEGER,
NAME VARCHAR(15),
LAT NUMERIC(4,1),
LONG NUMERIC(4,1),
SPEED SMALLINT,
PRESSURE INTEGER,
CATEGORY VARCHAR(2),
BASIN VARCHAR(16),
SHAPE_LENGTH NUMERIC(8,6)
)
"""

In [12]:
cur.execute(s)

In [13]:
file = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
read = csv.reader(io.TextIOWrapper(file))
next(read)

['\ufeffFID',
 'YEAR',
 'MONTH',
 'DAY',
 'AD_TIME',
 'BTID',
 'NAME',
 'LAT',
 'LONG',
 'WIND_KTS',
 'PRESSURE',
 'CAT',
 'BASIN',
 'Shape_Leng']

In [14]:
row = []
for l in read:
    t_r = datetime(int(l[1]), int(l[2]), int(l[3]), 
                   hour = int(l[4][:2]), 
                   minute = int(l[4][2:-1]))
    line = [l[0], t_r] + l[5:]
    row.append(
    cur.mogrify(
    "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
    line).decode('utf-8'))
s = 'INSERT INTO data VALUES ' + ",".join(row)


In [15]:
cur.execute(s)

# Permissions

In [16]:
cur.execute('REVOKE ALL ON data FROM vipul')

In [17]:
cur.execute('REVOKE ALL on data FROM other')

In [18]:
cur.execute('GRANT ALL ON data TO vipul')

In [19]:
cur.execute('GRANT SELECT ON data to other')


In [20]:
conn.close()